In [8]:
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import copy
import torch as tr

# import scripts
from src.es import random_opt
from src.es import evolutionary_strategy
from src.grad_opt import ADAM_opt
from src.api_helper import api_utils
from src.api_helper import env
import copyreg

device = tr.device("cuda" if tr.cuda.is_available() else "cpu")
dtype = tr.float32

##### data

In [9]:
## multi-process has problem with decorator
# def af(query): return env.rosenbrock(query); api = api_utils.multi_process_transform(af) 

@api_utils.transform
def api(query): return env.rosenbrock(query)

# @api_utils.multi_thread_transform
# def api(query): return env.rosenbrock(query)

x0, y0 = tr.tensor([4,4], dtype=dtype).view(1, -1), tr.tensor([[-1]], dtype=dtype).view(-1, 1)  
r0 = env.rosenbrock(x0)  # initial reward on which normalisation based

x0, y0= x0.to(device), y0.to(device)  # data in GPU

print("init query:", x0); print(f"init reward: {api(x0, r0, device)}, origin reward: {r0.item():.2f}")  # x,y stay on GPU while r0 in cpu



init query: tensor([[4., 4.]])
init reward: tensor([[-1.]]), origin reward: 14409.00


##### OPT

In [10]:
# hyperparameters
T = 20  # total number of iterations

adam = ADAM_opt(); ram_opt = random_opt(); es = evolutionary_strategy()

In [11]:
%%time
x,y = es.outer_loop(T, x0, r0, api)

CPU times: user 238 ms, sys: 3.41 ms, total: 241 ms
Wall time: 270 ms


In [17]:
# tr.save((x, y, "20_def_es"), "./data/20_def_es.pt")

In [19]:
%%time
x,y = ram_opt.outer_loop(T, (-4,4), x0, r0, api, batch_size=2)

CPU times: user 14.6 ms, sys: 2.16 ms, total: 16.7 ms
Wall time: 17.6 ms


In [ ]:
# tr.save((x, y, "20_random"), "./data/20_random.pt")

In [20]:
%%time
x1,y1 = adam.outer_loop(T, x0, r0, api, env.rosenbrock_grad)

CPU times: user 19.9 ms, sys: 966 µs, total: 20.8 ms
Wall time: 22.5 ms


In [21]:
# tr.save((x1, y1, "20_def_adam"), "./data/20_def_adam.pt")